In [1]:
# Import Libiary
import pandas as pd
import numpy as np
from glob import glob
from datetime import datetime,timedelta
from dateutil.relativedelta import relativedelta , TH
import warnings
warnings.filterwarnings('ignore')

In [13]:
# Strategy Rules
# Entry Time 9:16 am
# price less then 65 CE and PE both sell same time
# Put Stop loss 131 both position
# if one of there Stop loss hit we will exit both postion
# re Entry after 60 sec 
# re entry 3 time 
# after 3:00 pm no re entry
# If not hit stop loss then Position Exit 3:20 pm

In [2]:
# Create trade log DataFrame 
trade_log = pd.DataFrame(columns = ['Date' ,'Entry_time', 'Attempt','Day_to_exp' ,'Ce_strike_price','Ce_entry_price','Ce_exit_price' ,'ce_exit_time','ce_pnl' ,'Pe_strike_price' ,'Pe_entry_price',
                                        'Pe_exit_price' ,'put_exit_time', 'pe_pnl','Info', 'pnl'])

In [3]:
trade_log.to_csv('E:/Report/Nifty/2023/trade_log_file_August_1_2023' , index = False)

In [4]:
# Create BackTesting Fancation
def backtesting(data_path,trade_file):
    entry_time = '09:14:59'
    exit_time = '15:19:59'
    no_reentry = '14:59:59'
    attempt = 1
    n=4
    while (entry_time < no_reentry and attempt !=n):
        try:
            print('Trade on')
            # Read Data
            df = pd.read_csv(data_path)
            trade_log = pd.read_csv(trade_file)
            
            # create Entry Time and Re Entry Time
            one_minute = timedelta(minutes=1)
            time =pd.to_datetime(entry_time)
            entry_time =time + one_minute
            entry_time = pd.to_datetime(entry_time).time()
            entry_time =str(entry_time)
            df['Date'] = pd.to_datetime(df['Date'])
            date = df['Date'].iloc[0]
            print(date)
            
            # Change Expiry Format
            df['Expiry'] = pd.to_datetime(df['Expiry'],format="%Y%m%d")
            
            # create nearest Expiry
            nearest_exp = df['Date'].apply(lambda x: x.date()+relativedelta(weekday = TH))
            
            # Separate Call Data
            data_ce = df[(df['OptionType'] == 'CE') & (df['Expiry'] ==nearest_exp)|(df['Expiry'] == nearest_exp - pd.DateOffset(days=1))|(df['Expiry'] == nearest_exp - pd.DateOffset(days=2)) &  (df['Time'] >= entry_time)].reset_index(drop = 'Frist')
            data_ce=data_ce[data_ce['Time'] >=entry_time].reset_index(drop = 'Frist')
            
            # Create Call Entry Price
            ce_entry_price = data_ce[(data_ce['Open'] <=65) & (data_ce['Time'] == entry_time)]['Open'].reset_index(drop = 'Frits').iloc[0]
            
            # Create Call Strike price
            ce_strike_price = data_ce[(data_ce['Open'] <=65) & (data_ce['Time'] == entry_time)]['Strike'].reset_index(drop = 'Frits').iloc[0]
            
            # Create Intrday Call data
            intrday_ce = data_ce[data_ce['Strike'] ==ce_strike_price].reset_index(drop = 'Frist')
            
            # Drop Unuse columns
            intrday_ce.drop(['Symbol','Expiry','Strike','OptionType' , 'Open','Low' , 'Close' , 'Day' , 'Month' , 'Year'],axis=1 ,inplace = True)
            intrday_ce['Ce_High'] = intrday_ce['High']
            intrday_ce.drop(['High'] , axis =1,inplace = True)
            
            # Separate Put Data
            data_pe = df[(df['OptionType'] == 'PE') & (df['Expiry'] ==nearest_exp)|(df['Expiry'] == nearest_exp - pd.DateOffset(days=1))|(df['Expiry'] == nearest_exp - pd.DateOffset(days=2)) &  (df['Time'] >= entry_time)].reset_index(drop = 'Frist')
            data_pe=data_pe[data_pe['Time'] >=entry_time].reset_index(drop = 'Frist')
            
            # Create Put Entry Price
            pe_entry_price =data_pe[(data_pe['Open'] <=65) & (data_pe['Time'] == entry_time)].reset_index(drop = 'Frits')['Open'].iloc[-1]
            
            # Create Put Strike Price
            pe_strike_price =data_pe[(data_pe['Open'] <=65) & (data_pe['Time'] == entry_time)].reset_index(drop = 'Frits')['Strike'].iloc[-1]
            intrday_pe = data_pe[data_pe['Strike'] ==pe_strike_price].reset_index(drop = 'Frist')
            
            # Create Put Intrday Data
            intrday_pe.drop(['Symbol','Expiry','Strike','OptionType' , 'Open','Low' , 'Close' , 'Day' , 'Month' , 'Year','Date','Time'],axis=1 ,inplace = True)
            
            # Drop Unuse Columns
            intrday_pe['Pe_High'] =intrday_pe['High']
            intrday_pe.drop(['High'] ,axis=1 ,inplace=True)
            intrday = pd.concat([intrday_ce,intrday_pe],axis=1)
            
            # Create BackTesting logic
            stop_loss = 131
            ce_exit_datetime = ''
            pe_exit_datetime =''
            ce_exit_price = 0
            pe_exit_price = 0
            ce_pnl = 0
            pe_pnl = 0
            pnl = 0
            status = 0
            for index,row in intrday.iterrows():
                #print(index)
                ce_ltp = row['Ce_High']
                pe_ltp = row['Pe_High']
                #1 Not hit stoploss any position
                if (row['Time'] == exit_time) & (status == 0):
                    ce_pnl = ce_entry_price - ce_ltp
                    pe_pnl = pe_entry_price - pe_ltp
                    status = 1
                    ce_exit_price = ce_ltp
                    pe_exit_price = pe_ltp
                    ce_exit_datetime = row['Time']
                    pe_exit_datetime = row['Time']
                    pnl = ce_pnl + pe_pnl
                    info = 'Not hit Stop loss Call and Put Both Position Exit 15:20'
                    print('Not hit Stop loss Call and Put Both Position Exit 15:20')
                    break
                    #2 call or put hit stop loss then both are position cut same time
                elif (ce_ltp >= stop_loss) or (pe_ltp >= stop_loss) & (status == 0):

                    ce_pnl = ce_entry_price - ce_ltp
                    pe_pnl = pe_entry_price - pe_ltp
                    status = 1
                    ce_exit_price = ce_ltp
                    pe_exit_price = pe_ltp
                    ce_exit_datetime = row['Time']
                    pe_exit_datetime = row['Time']
                    pnl = ce_pnl + pe_pnl
                    info = 'Stop loss hit Both Position exit Make again Position'
                    print('Stop loss hit Both Position exit Make again Position')
                    break
            
            # Append Result in Trade log
            trade_log = trade_log.append({'Date':date , 'Entry_time': entry_time,'Attempt':attempt,'Day_to_exp':((pd.to_datetime(nearest_exp[0]).day) -(pd.to_datetime(date).day)) , 'Ce_entry_price':ce_entry_price , 'Pe_entry_price':pe_entry_price , 'Ce_strike_price':ce_strike_price ,

                                                  'Pe_strike_price':pe_strike_price , 'Ce_exit_price' :ce_exit_price, 'Pe_exit_price':pe_exit_price , 'ce_pnl':ce_pnl , 'pe_pnl':pe_pnl , 'ce_exit_time':ce_exit_datetime , 'put_exit_time':ce_exit_datetime ,'Info':info, 'pnl':pnl},ignore_index=True)
            # Save trade log in csv format

            trade_log.to_csv('E:/Report/Nifty/2023/trade_log_file_August_1_2023' , index=False)
            
            # Read Trade log File
            dt = pd.read_csv(r'E:\Report\Nifty\2023\trade_log_file_August_1_2023')
            # Define Re entry Time
            entry_time=dt['ce_exit_time'].iloc[-1]
            attempt = attempt + 1
        except Exception as e:
            print(e)
            break

In [5]:
# Data Files
data = pd.DataFrame(glob(r'E:\Option_data\Nifty\2023\August/*'),columns = ['Location'])

In [6]:
data

,Location
0,E:\Option_data\Nifty\2023\August\option_data_1
1,E:\Option_data\Nifty\2023\August\option_data_2
2,E:\Option_data\Nifty\2023\August\option_data_3
3,E:\Option_data\Nifty\2023\August\option_data_7
4,E:\Option_data\Nifty\2023\August\option_data_8
5,E:\Option_data\Nifty\2023\August\option_data_9
6,E:\Option_data\Nifty\2023\August\option_data_10
7,E:\Option_data\Nifty\2023\August\option_data_4


In [7]:
# Input Data and Trade log file in Backtesting Fancation
file_log =r'E:\Report\Nifty\2023\trade_log_file_July_1_2023'
for file in data['Location']:
    backtesting(file,file_log)

Trade on
2023-08-01 00:00:00
Not hit Stop loss Call and Put Both Position Exit 15:20
Trade on
2023-08-02 00:00:00
Stop loss hit Both Position exit Make again Position
Trade on
2023-08-02 00:00:00
Stop loss hit Both Position exit Make again Position
Trade on
2023-08-02 00:00:00
Not hit Stop loss Call and Put Both Position Exit 15:20
Trade on
2023-08-03 00:00:00
Stop loss hit Both Position exit Make again Position
Trade on
2023-08-03 00:00:00
Not hit Stop loss Call and Put Both Position Exit 15:20
Trade on
2023-08-07 00:00:00
Not hit Stop loss Call and Put Both Position Exit 15:20
Trade on
2023-08-08 00:00:00
Not hit Stop loss Call and Put Both Position Exit 15:20
Trade on
2023-08-09 00:00:00
Not hit Stop loss Call and Put Both Position Exit 15:20
Trade on
2023-08-10 00:00:00
Stop loss hit Both Position exit Make again Position
Trade on
2023-08-10 00:00:00
Not hit Stop loss Call and Put Both Position Exit 15:20
Trade on
2023-08-04 00:00:00
Not hit Stop loss Call and Put Both Position Exi

In [11]:
# Result
Result =pd.read_csv(r'E:\Report\Nifty\2023\trade_log_file_July_1_2023')

In [12]:
Result

,Date,Entry_time,Attempt,Day_to_exp,Ce_strike_price,Ce_entry_price,Ce_exit_price,ce_exit_time,ce_pnl,Pe_strike_price,Pe_entry_price,Pe_exit_price,put_exit_time,pe_pnl,Info,pnl
0,2023-07-03,09:15:59,1,3,19300,46.00,90.05,15:19:59,-44.05,19150,49.00,25.85,15:19:59,23.15,Not hit Stop loss Call and Put Both Position E...,-20.90
1,2023-07-04 00:00:00,09:15:59,1,2,19400,50.00,52.90,15:19:59,-2.90,19350,62.30,45.25,15:19:59,17.05,Not hit Stop loss Call and Put Both Position E...,14.15
2,2023-07-05 00:00:00,09:15:59,1,1,19400,52.05,49.25,15:19:59,2.80,19400,64.90,49.20,15:19:59,15.70,Not hit Stop loss Call and Put Both Position E...,18.50
3,2023-07-06 00:00:00,09:15:59,1,0,19350,62.20,136.75,14:18:59,-74.55,19400,53.55,4.50,14:18:59,49.05,Stop loss hit Both Position exit Make again Po...,-25.50
4,2023-07-06 00:00:00,14:19:59,2,0,19450,44.15,53.00,15:19:59,-8.85,19500,36.25,1.80,15:19:59,34.45,Not hit Stop loss Call and Put Both Position E...,25.60
5,2023-07-07 00:00:00,09:15:59,1,6,19550,64.40,22.60,15:19:59,41.80,19300,62.10,75.90,15:19:59,-13.80,Not hit Stop loss Call and Put Both Position E...,28.00
6,2023-07-11 00:00:00,09:15:59,1,2,19450,55.00,58.30,15:19:59,-3.30,19350,50.00,18.40,15:19:59,31.60,Not hit Stop loss Call and Put Both Position E...,28.30
7,2023-07-13 00:00:00,09:15:59,1,0,19450,50.50,0.25,15:19:59,50.25,19450,20.10,37.80,15:19:59,-17.70,Not hit Stop loss Call and Put Both Position E...,32.55
8,2023-07-10 00:00:00,09:15:59,1,3,19450,50.55,38.65,15:19:59,11.90,19300,50.00,46.15,15:19:59,3.85,Not hit Stop loss Call and Put Both Position E...,15.75
9,2023-07-12 00:00:00,09:15:59,1,1,19500,44.00,15.85,15:19:59,28.15,19500,61.95,115.05,15:19:59,-53.10,Not hit Stop loss Call and Put Both Position E...,-24.95
